# Dijkstra's with all possible s-t paths

This code will present the user with the shortest path and all possible s-t paths

In [124]:
import random
from igraph import Graph

In [125]:
#takes input of graph and m value for randomized arc lengths
def create_graph():
    #input of nodes
    node_connections = []
    #splitting and uppercasing nodes input
    nodes = tuple(str(input("Please enter all nodes (Comma seperated, e.g., 'S, E, G, H, U, T')\nYour Input: ")).split(', '))
    print(f"All nodes: {nodes}")
    #loop through nodes and get their connections
    for n in nodes:
        if n != 'T':
            connections = str(input(f"Please enter all nodes that node {n} connects to (Comma seperated, e.g., 'H, G, T')\nYour Input: ")).split(', ')
            node_connections.append(connections)
        else:
            break    
    for i in range(len(nodes)-1):
        print(f"Node {nodes[i]} connects to {node_connections[i]}") 
    #getting M value (upper bound of randomized arc weights)
    m = int(input("Please Enter Your m Value(upper bound of randomized arc weights): "))
    
    #storing dict representation of connections
    all_conns = []
    #loop through node connections and create temp dicts with node:weight
    for n in node_connections:
        temp_dict = {}
        for i in n:
            temp_dict[i] = random.randint(1, m)
        all_conns.append(temp_dict)
    
    #creation of graph dict
    graph = {}
    for i in range(len(nodes)-1):
        graph[nodes[i]] = all_conns[i]
    graph['T'] = {}
    return graph

In [135]:
#conversion to igraph to present shortest graph and all s-t paths possible
def solve_graph(graph):
    #retrieving nodes, edges and weights
    labels = [i for i in graph.keys()]
    #getting weights of arcs in order for edges
    nested_weights = [[i for i in j.values()] for j in graph.values()]
    #flattening nested list
    weights = [weight for i in nested_weights for weight in i]
    #need to loop through dict, pairing key with subkeys in tuples - e.g., (S, E), (S, G)
    edgs = [[(i[0], j) for j in i[1].keys()] for i in graph.items()]
    pre_edges = [edge for i in edgs for edge in i]
    #encode vertices from 0-len(nodes)
    vertex_ids = {labels[i]: i for i in range(len(labels))}
    edges = []
    for edge in pre_edges:
        edges.append((vertex_ids[edge[0]], vertex_ids[edge[1]]))
    #Create graph
    g = Graph(directed=True)

    #add vertices
    g.add_vertices(len(labels))
    
    #add edges between nodes
    g.add_edges(edges)

    #adding ids and labels to vertices
    for i in range(len(g.vs)):
        g.vs[i]['id']=i
        g.vs[i]['label']=labels[i]

    #adding weights and edge labels
    g.es['weight'] = weights
    g.es['label'] = weights
    
    #getting all s-t paths
    all_paths = g.get_all_simple_paths(0, 6)
    
    #shortest path with built in dijkstra's
    shortest = g.get_shortest_paths(0, 6)
    
    #showing shortest path retrieved from Dijkstra's
    print(f"Shortest S-T Path: {convert(shortest[0], labels)}")
    
    #showing all possible s-t paths
    print(f"All possible S-T paths: {[convert(i, labels) for i in all_paths]}")

In [132]:
#function that takes a path as integers and returns as nodes
def convert(path, nodes):
    encoding = {i:nodes[i] for i in range(len(nodes))}
    new_path = [encoding[i] for i in path]
    return f"Path: {', '.join(new_path)}"

In [128]:
graph = create_graph()

Please enter all nodes (Comma seperated, e.g., 'S, E, G, H, U, T')
Your Input: S, E, G, W, J, K, T
All nodes: ('S', 'E', 'G', 'W', 'J', 'K', 'T')
Please enter all nodes that node S connects to (Comma seperated, e.g., 'H, G, T')
Your Input: E, G
Please enter all nodes that node E connects to (Comma seperated, e.g., 'H, G, T')
Your Input: W, J
Please enter all nodes that node G connects to (Comma seperated, e.g., 'H, G, T')
Your Input: J
Please enter all nodes that node W connects to (Comma seperated, e.g., 'H, G, T')
Your Input: K, T
Please enter all nodes that node J connects to (Comma seperated, e.g., 'H, G, T')
Your Input: K
Please enter all nodes that node K connects to (Comma seperated, e.g., 'H, G, T')
Your Input: T
Node S connects to ['E', 'G']
Node E connects to ['W', 'J']
Node G connects to ['J']
Node W connects to ['K', 'T']
Node J connects to ['K']
Node K connects to ['T']
Please Enter Your m Value(upper bound of randomized arc weights): 50


In [137]:
solve_graph(graph)

Shortest S-T Path: Path: S, E, W, T
All possible S-T paths: ['Path: S, E, W, K, T', 'Path: S, E, W, T', 'Path: S, E, J, K, T', 'Path: S, G, J, K, T']
